In [ ]:
!pip install qsimov numpy loguru

In [1]:
import numpy as np
import cProfile
import os
os.environ['LOGURU_LEVEL']="INFO"

In [6]:
import qasimod
reg = qasimod.Register(5)
reg.x([0,1,2])
reg.apply_now()
reg.visualize(only_ones=True)

current_state = tuple(range(5))
desired_state = tuple([1,3,4,0,2])

2024-05-10 05:29:08.689 | INFO     | qasimod:apply_now:111 - Applying 3 gates
2024-05-10 05:29:08.690 | INFO     | qasimod:apply_swaps:119 - Applying 0 swaps
2024-05-10 05:29:08.692 | INFO     | qasimod:apply_swaps:119 - Applying 0 swaps


State     Value     
|00111>   1.0       


In [7]:
reg.cccci([1,3,4,0,2])
reg.apply_now()
reg.visualize(only_ones=True)

2024-05-10 05:29:10.338 | INFO     | qasimod:apply_now:111 - Applying 1 gates
2024-05-10 05:29:10.340 | INFO     | qasimod:apply_swaps:119 - Applying 3 swaps
2024-05-10 05:29:10.342 | INFO     | qasimod:apply_swaps:119 - Applying 3 swaps


State     Value     
|00111>   1.0       


In [8]:
swaps = qasimod.obtain_swaps(current_state, desired_state)
swap_block = np.eye(1 << len(current_state))
for swap in swaps:
    reg.state = qasimod.compute_swap_gate(*swap, len(current_state)) @ reg.state
    reg.visualize(only_ones=True)

State     Value     
|00111>   1.0       
State     Value     
|01101>   1.0       
State     Value     
|11001>   1.0       


In [3]:
from PyQSim import Register
import cProfile
import numpy as np
rng = np.random.default_rng()

def test():
    reg = Register(12)
    reg.h(list(range(12)))
    for _ in range(10):
        control = rng.integers(12)
        while (target := rng.integers(12)) == control:
            continue
        reg.cx(control,target)
        reg.apply_now()
    reg.apply_now()

    # reg.visualize()

cProfile.run('test()')

2024-05-11 03:06:58.042 | INFO     | PyQSim.Register:apply_now:76 - Applying 12 gates
2024-05-11 03:06:58.044 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.252 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.299 | INFO     | PyQSim.Register:apply_now:76 - Applying 1 gates
2024-05-11 03:06:58.300 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.428 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.475 | INFO     | PyQSim.Register:apply_now:76 - Applying 1 gates
2024-05-11 03:06:58.477 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.628 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.676 | INFO     | PyQSim.Register:apply_now:76 - Applying 1 gates
2024-05-11 03:06:58.677 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:06:58.873 | INFO     | PyQSim.Regist

         10277 function calls (9869 primitive calls) in 43.659 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   43.659   43.659 3618540124.py:6(test)
       17    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
       17    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
      204    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(expand_dims)
       56    0.000    0.000    0.242    0.004 <__array_function__ internals>:177(kron)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(where)
        1    0.000    0.000   43.659   43.659 <string>:1(<module>)
        5    0.300    0.060    0.542    0.108 Register.py:107(_apply_gates)
        5    0.000    0.000    0.251    0.050 Register.py:127(_reverse_swaps)
        1    0.000    0.000    0.000    0.000 Register.py:13(__init__)
        1    0.000    0

KeyboardInterrupt: 

In [ ]:
import numpy as np

def mask_generator_string(bit, length):
    mask= np.array([int(i) for i in f"{1 << bit:0{length}b}"], dtype=bool)
    antimask = mask ^ np.ones(length, dtype=bool)
    return mask, antimask

def mask_generator_not_string(bit, length):
    mask = np.zeros(length, dtype=bool)
    mask[bit] = 1
    antimask = mask ^ np.ones(length, dtype=bool)
    return mask, antimask

def mask_generator_integers(bit, length):
    mask = 1 << bit
    antimask = ((1<<length)-1) ^ mask
    return mask, antimask

bit = 3
length = 5
print("mask_generator_string")

mask, antimask = mask_generator_string(bit, length)
display(mask)
display(antimask)

print("mask_generator_not_string")
mask, antimask = mask_generator_not_string(bit, length)
display(mask)
display(antimask)

print("mask_generator_integers")
mask, antimask = mask_generator_integers(bit, length)
display(mask)
display(antimask)


In [ ]:
import cProfile


print("mask_generator_string")
cProfile.run('_ = [mask_generator_string(bit,length) for i in range(100000)]')


In [ ]:
print("mask_generator_not_string")
cProfile.run('_ = [mask_generator_not_string(bit,length) for i in range(100000)]')

In [ ]:
print("mask_generator_integers")
cProfile.run('_ = [mask_generator_integers(bit,length) for i in range(100000)]')

In [ ]:
np.kron(np.kron(1, np.array([[1,1],[1,-1]])),np.array([[1,1,0],[1,-1,1],[1,-1,1]]))

In [ ]:
a ={1,2,4}
b = {1,2}
b.issubset(a)

In [ ]:
a = [0,1,2,3,4,5]
b = [3,1,4,0,5,2]

newb=a.copy()
for swap in zip(a,np.argsort(b)):
    print(swap)
    newb[swap[1]] = a[swap[0]]
    print(newb)

In [1]:
from PyQSim import Register

reg = Register(1)
reg.h([0])
reg.apply_now()
reg.bloch_sphere

2024-05-11 03:01:35.991 | INFO     | PyQSim.Register:apply_now:76 - Applying 1 gates
2024-05-11 03:01:35.992 | DEBUG    | PyQSim.Register:apply_now:77 - Applying {0: (<PyQSim.gates.Gate object at 0x7ff86decc910>, [0])}
2024-05-11 03:01:35.992 | DEBUG    | PyQSim.Register:_prepare_to_apply:98 - Transforming (0,) into (0,)
2024-05-11 03:01:35.993 | DEBUG    | PyQSim.compute.swap:obtain_swaps:30 - Desired state: [0]
2024-05-11 03:01:35.994 | DEBUG    | PyQSim.Register:_prepare_to_apply:104 - Preparation swaps: ()
2024-05-11 03:01:35.995 | INFO     | PyQSim.Register:apply_swaps:84 - Applying 0 swaps
2024-05-11 03:01:35.995 | DEBUG    | PyQSim.Register:apply_swaps:85 - ()
2024-05-11 03:01:35.996 | DEBUG    | PyQSim.Register:_apply_gates:121 - Buffer
2024-05-11 03:01:35.997 | DEBUG    | PyQSim.Register:_apply_gates:122 - [[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
2024-05-11 03:01:35.998 | DEBUG    | PyQSim.Register:_apply_gates:123 - state
2024-05-11 03:01:35.998 | DEBUG    | PyQ

(array([0.]), array([1.57079633]))